In [ ]:
import os
import json
import pandas as pd

feature_type = 'origin'
root_dir = f'Results_zhongda_before_after_{feature_type}'

project_dir = f'../{root_dir}/'

# 初始化存储结果的列表
results = []

# 遍历项目目录下的所有文件夹
for folder in os.listdir(project_dir):
    folder_path = os.path.join(project_dir, folder)
    if os.path.isdir(folder_path):
        info_path = os.path.join(folder_path, '00_info.json')
        if os.path.exists(info_path):
            # 读取info.json文件
            with open(info_path, 'r') as f:
                data = json.load(f)
                roc_auc_list = data.get('roc_auc_list', [])
                prc_auc_list = data.get('prc_auc_list', [])

                # 确保列表不为空
                if roc_auc_list and prc_auc_list:
                    # 计算roc_auc_list中的最大值及其索引
                    max_roc_auc = max(roc_auc_list)
                    max_roc_index = roc_auc_list.index(max_roc_auc)
                    corresponding_prc_for_max_roc = prc_auc_list[max_roc_index]

                    # 计算prc_auc_list中的最大值及其索引
                    max_prc_auc = max(prc_auc_list)
                    max_prc_index = prc_auc_list.index(max_prc_auc)
                    corresponding_roc_for_max_prc = roc_auc_list[max_prc_index]

                    # 计算roc_auc_list和prc_auc_list乘积的最大值及其索引
                    auc_prc_product_list = [roc * prc for roc, prc in zip(roc_auc_list, prc_auc_list)]
                    max_product = max(auc_prc_product_list)
                    max_product_index = auc_prc_product_list.index(max_product)
                    roc_at_max_product = roc_auc_list[max_product_index]
                    prc_at_max_product = prc_auc_list[max_product_index]
                    
                    # 计算roc_auc_list和prc_auc_list和的最大值及其索引
                    auc_prc_plus_list = [(roc + prc) for roc, prc in zip(roc_auc_list, prc_auc_list)]
                    max_plus = max(auc_prc_plus_list)
                    max_plus_index = auc_prc_plus_list.index(max_plus)
                    roc_at_max_plus = roc_auc_list[max_plus_index]
                    prc_at_max_plus = prc_auc_list[max_plus_index]
# ZYY_BiLSTM_BN_3layers_model_undersample_FocalLoss_100_1e-05
                    # 存储结果
                    results.append([
                        folder, 
                        max_roc_auc, 
                        max_roc_index + 1,  # 转换为从1开始的索引
                        corresponding_prc_for_max_roc, 
                        max_prc_auc, 
                        max_prc_index + 1,  # 转换为从1开始的索引
                        corresponding_roc_for_max_prc,
                        max_product,
                        roc_at_max_product,
                        prc_at_max_product,
                        max_product_index + 1,  # 转换为从1开始的索引
                        max_plus,
                        roc_at_max_plus,
                        prc_at_max_plus,
                        max_plus_index + 1  # 转换为从1开始的索引
                    ])

# 转换结果为DataFrame并输出表格
df = pd.DataFrame(results, columns=[
    'Folder', 
    'Max ROC AUC', 
    'ROC Index', 
    'PRC at Max ROC', 
    'Max PRC AUC', 
    'PRC Index', 
    'ROC at Max PRC',
    'Max AUC*PRC Product',
    'ROC at Max Product',
    'PRC at Max Product',
    'Product Index',
    'Max AUC+PRC Plus',
    'ROC at Max Plus',
    'PRC at Max Plus',
    'Plus Index'
])

# 如果需要保存为CSV文件，可以使用以下代码
output_path = os.path.join(f'auc_results_{feature_type}.csv')
df.to_csv(output_path, index=False)

In [5]:
df

,Folder,Max ROC AUC,ROC Index,PRC at Max ROC,Max PRC AUC,PRC Index,ROC at Max PRC,Max AUC*PRC Product,ROC at Max Product,PRC at Max Product,Product Index,Max AUC+PRC Plus,ROC at Max Plus,PRC at Max Plus,Plus Index
0,use_20_predict_20_BiLSTM_BN_3layers_model_unde...,0.820765,50,0.223529,0.225104,37,0.812305,0.183464,0.820765,0.223529,50,1.044294,0.820765,0.223529,50
1,use_20_predict_20_BiLSTM_BN_4layers_model_unde...,0.812710,9,0.205375,0.206095,5,0.802418,0.166910,0.812710,0.205375,9,1.018084,0.812710,0.205375,9
2,use_20_predict_20_BiLSTM_BN_larger_model_under...,0.815909,24,0.196290,0.210020,49,0.810852,0.170295,0.810852,0.210020,49,1.020872,0.810852,0.210020,49
3,use_20_predict_20_BiLSTM_BN_model_undersample_...,0.807817,48,0.205801,0.206484,50,0.806088,0.166444,0.806088,0.206484,50,1.013618,0.807817,0.205801,48
4,use_20_predict_20_BiLSTM_BN_Resnet_model_under...,0.808946,42,0.195163,0.198007,49,0.808595,0.160108,0.808595,0.198007,49,1.006602,0.808595,0.198007,49


In [ ]:
delta = df[['Folder', 'Max AUC+PRC Plus', 'ROC at Max Plus', 'PRC at Max Plus']].sort_values(by=['Max AUC+PRC Plus'], ascending=[False])
delta.head()

In [ ]:
with open('../生成tensor/delta/mice_mmscaler_use_20_predict_24_ids_delta.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
print(f"1h Merge info")
print(f"Num of train EW_IDs: {len(data['ids_train_data'])}")
print(f"Num of valid EW_IDs: {len(data['ids_valid_data'])}")
print(f"Num of test EW_IDs: {len(data['ids_test_data'])}")
import torch
for typeee in ['train', 'val', 'test'] :
    label = torch.load('../生成tensor/delta/mice_mmscaler_use_20_predict_24_delta.pth')[f'label_tensor_{typeee}'].numpy()
    num = sum(label)/len(label)
    print(f"Num of {typeee} Event%: {int(num*10000)/100}% = {int(sum(label))} / {len(label)}")